In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from src.config import MODELS_DIR

pd.set_option('display.max_columns', None)

2025-04-17 00:41:05.335 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /Users/mayankanand/Documents/LnT_PearlChallenge/income_prediction


In [2]:
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = DATA_DIR.parent / 'processed'

In [3]:
! ls $PROCESSED_DIR

exp_v0 exp_v1


# Data

In [4]:
exp_name="exp_v1"
train_fname="train_v0.csv"
test_fname="test_v0.csv"

In [5]:
train_data_path = PROCESSED_DIR / exp_name / train_fname
train_df = pd.read_csv(train_data_path)

In [6]:
test_data_path = PROCESSED_DIR / exp_name / test_fname
test_df = pd.read_csv(test_data_path)

test_df.head()

,FarmerID,perc_of_house_with_6plus_room,women_15_19_mothers_or_pregnant_at_time_of_survey,perc_of_pop_living_in_hh_electricity,perc_households_with_pucca_house_that_has_more_than_3_rooms,mat_roof_metal_gi_asbestos_sheets,perc_of_wall_material_with_burnt_brick,households_with_improved_sanitation_facility,perc_households_do_not_have_kcc_with_the_credit_limit_of_50k,total_geographical_area_in_hectare,net_agri_area_in_hectare,perc_net_agri_area_in_hectare,land_holding_index_source,road_density,avg_disbursement_amount_bureau,kharif_seasons_cropping_density_2022,rabi_seasons_cropping_density_2022,kharif_seasons_agricultural_performance_2022,rabi_seasons_agricultural_performance_2022,kharif_seasons_agricultural_score_2022,rabi_seasons_agricultiral_score_2022,kharif_seasons_irrigated_area_2022,rabi_seasons_irrigated_area_2022,kharif_season_avg_groundwater_thickness_cm_2022,rabi_season_avg_groundwater_thickness_cm_2022,kharif_season_avg_groundwater_replenishment_rate_cm_2022,rabi_season_avg_groundwater_replenishment_rate_cm_2022,kharif_season_avg_rainfall_mm_2022,rabi_season_avg_rainfall_mm_2022,kharif_average_rainfall_2022_2021_growth,rabi_average_rainfall_2022_2021_growth,rabi_average_rainfall_2021_2020_growth,kharif_season_groundwater_replenishment_rate_2022_2021_growth,rabi_season_groundwater_replenishment_rate_2022_2021_growth,kharif_season_groundwater_replenishment_rate_2021_2020_growth,rabi_season_groundwater_replenishment_rate_2021_2020_growth,kharif_groundwater_thickness_2022_2021_growth,rabi_groundwater_thickness_2022_2021_growth,kharif_groundwater_thickness_2021_2020_growth,rabi_groundwater_thickness_2021_2020_growth,kharif_season_agricultural_performance_2022_2021_growth,rabi_season_agricultural_performance_2022_2021_growth,kharif_season_agricultural_performance_2021_2020_growth,rabi_season_agricultural_performance_2021_2020_growth,kharif_season_agriculture_score_2022_2021_growth,rabi_season_agriculture_score_2022_2021_growth,kharif_season_agriculture_score_2021_2020_growth,rabi_season_agriculture_score_2021_2020_growth,kharif_ambient_temperature_min_2022,kharif_ambient_temperature_max_2022,rabi_ambient_temperature_min_2022,rabi_ambient_temperature_max_2022,kharif_ambient_temperature_min_2021,kharif_ambient_temperature_max_2021,rabi_ambient_temperature_min_2021,rabi_ambient_temperature_max_2021,rabi_ambient_temperature_min_2020,rabi_ambient_temperature_max_2020,state,region,soil_type,kharif_vill_cat_on_agri_params_2022,rabi_vill_cat_on_agri_params_2022,village_cat_on_socio_economic_params,waterbody_reservoir,waterbody_river,waterbody_riverbank,waterbody_water,waterbody_wetland,ecological_subzone,village_en_name,district_en_name,no_of_active_loan_in_bureau,non_agriculture_income,active_loan_bins,village_non_agri_income_bin,village_total_land_for_agriculture_bin,district_non_agri_bin,district_total_land_for_agriculture_bin
0,576972022499073,1.27,4.7,99.6,6.92,22.59,62.68,86.1,0.63,980.33,7.887,0.80,0.000118,8.85,0.0,54.17,8.59,0.17,3.93,2.67,18.52,8.82,88.66,101.75,104.67,19.91,30.01,1118.77,360.14,0.354398,0.473094,0.394446,-0.087895,0.369877,-0.193906,-0.234267,0.054349,0.077959,0.024527,0.043104,-42.703370,-0.821881,-0.162853,-0.294692,-19.85001,-21.48001,-21.84001,-21.95001,19.92,26.84,18.22,28.54,20.30,26.86,17.82,28.77,18.92,29.60,8,3,6,1,1,1,0,0,0,1,0,9,2137,330,0,315000,0,4,0,4,0
1,979235081831136,6.29,1.4,99.5,15.05,3.35,90.96,82.6,0.24,92.09,10.340,11.23,0.000268,43.23,400000.0,56.09,0.85,1.68,4.26,7.59,2.04,25.44,1.02,49.11,38.65,20.68,11.58,953.80,320.01,-0.187209,-0.034374,-1.282469,0.446325,0.782383,0.000873,-4.067444,0.093464,0.048383,0.069182,-0.193855,0.999994,0.084507,0.000000,0.999997,-0.03001,-2.14001,-0.03001,-0.03001,21.68,29.42,11.00,22.29,21.83,29.08,11.05,22.88,10.30,20.74,6,2,0,1,1,1,0,0,0,0,0,1,1124,246,1,0,1,0,1,0,1
2,176490610549774,6.01,1.4,99.5,15.05,3.52,90.58,82.6,0.24,92.09,10.340,11.23,0.000268,43.23,0.0,56.09,0.85,1.68,4.26,7.59,2.04,25.44,1.02,49.11,38.65,20.68,11.58,953.80,320.01,-0.18720

In [7]:
train_df.head()

,FarmerID,perc_of_house_with_6plus_room,women_15_19_mothers_or_pregnant_at_time_of_survey,perc_of_pop_living_in_hh_electricity,perc_households_with_pucca_house_that_has_more_than_3_rooms,mat_roof_metal_gi_asbestos_sheets,perc_of_wall_material_with_burnt_brick,households_with_improved_sanitation_facility,perc_households_do_not_have_kcc_with_the_credit_limit_of_50k,total_geographical_area_in_hectare,net_agri_area_in_hectare,perc_net_agri_area_in_hectare,land_holding_index_source,road_density,avg_disbursement_amount_bureau,kharif_seasons_cropping_density_2022,rabi_seasons_cropping_density_2022,kharif_seasons_agricultural_performance_2022,rabi_seasons_agricultural_performance_2022,kharif_seasons_agricultural_score_2022,rabi_seasons_agricultiral_score_2022,kharif_seasons_irrigated_area_2022,rabi_seasons_irrigated_area_2022,kharif_season_avg_groundwater_thickness_cm_2022,rabi_season_avg_groundwater_thickness_cm_2022,kharif_season_avg_groundwater_replenishment_rate_cm_2022,rabi_season_avg_groundwater_replenishment_rate_cm_2022,kharif_season_avg_rainfall_mm_2022,rabi_season_avg_rainfall_mm_2022,kharif_average_rainfall_2022_2021_growth,rabi_average_rainfall_2022_2021_growth,rabi_average_rainfall_2021_2020_growth,kharif_season_groundwater_replenishment_rate_2022_2021_growth,rabi_season_groundwater_replenishment_rate_2022_2021_growth,kharif_season_groundwater_replenishment_rate_2021_2020_growth,rabi_season_groundwater_replenishment_rate_2021_2020_growth,kharif_groundwater_thickness_2022_2021_growth,rabi_groundwater_thickness_2022_2021_growth,kharif_groundwater_thickness_2021_2020_growth,rabi_groundwater_thickness_2021_2020_growth,kharif_season_agricultural_performance_2022_2021_growth,rabi_season_agricultural_performance_2022_2021_growth,kharif_season_agricultural_performance_2021_2020_growth,rabi_season_agricultural_performance_2021_2020_growth,kharif_season_agriculture_score_2022_2021_growth,rabi_season_agriculture_score_2022_2021_growth,kharif_season_agriculture_score_2021_2020_growth,rabi_season_agriculture_score_2021_2020_growth,kharif_ambient_temperature_min_2022,kharif_ambient_temperature_max_2022,rabi_ambient_temperature_min_2022,rabi_ambient_temperature_max_2022,kharif_ambient_temperature_min_2021,kharif_ambient_temperature_max_2021,rabi_ambient_temperature_min_2021,rabi_ambient_temperature_max_2021,rabi_ambient_temperature_min_2020,rabi_ambient_temperature_max_2020,state,region,soil_type,kharif_vill_cat_on_agri_params_2022,rabi_vill_cat_on_agri_params_2022,village_cat_on_socio_economic_params,waterbody_reservoir,waterbody_river,waterbody_riverbank,waterbody_water,waterbody_wetland,ecological_subzone,village_en_name,district_en_name,no_of_active_loan_in_bureau,non_agriculture_income,active_loan_bins,village_non_agri_income_bin,village_total_land_for_agriculture_bin,district_non_agri_bin,district_total_land_for_agriculture_bin,Target_Variable/Total Income
0,1002818465057450,1.23,1.70,99.30,2.22,0.74,35.43,69.60,15.10,715.70,483.60,67.57,0.773129,0.00,0.000000,62.55,73.19,27.81,18.65,28.60,24.39,57.32,74.50,99.99,92.64,28.84,14.38,1633.97,81.26,0.416960,-0.226557,-1.798836,0.237171,-0.235049,-0.135909,-0.097973,0.065807,0.011982,-0.057810,-0.062384,0.724200,-0.204826,-0.629725,-0.513128,-18.62001,-34.37001,-22.69001,-16.38001,23.34,30.33,16.19,29.07,23.29,29.78,16.48,30.02,16.08,28.09,9,0,2,1,1,2,0,0,0,1,0,0,1968,273,0,100000,0,1,2,1,3,1360000
1,1003667148159900,1.77,7.70,98.60,3.97,0.34,11.44,53.70,7.17,492.38,391.84,79.58,0.199800,0.00,0.000000,69.89,85.72,36.57,43.73,33.04,37.27,60.44,81.43,96.35,93.33,21.91,13.36,1584.60,132.10,0.323116,-1.039288,0.654627,-0.124601,0.010479,0.206169,-0.565053,0.017436,0.017358,0.009824,-0.039254,0.452557,0.094901,0.022977,0.520717,-24.88001,-50.67001,-33.46001,-26.11001,23.70,31.03,14.93,27.92,23.88,30.97,15.17,28.53,14.98,26.93,9,0,2,0,0,0,0,0,0,0,0,0,0,19,0,40000,0,1,3,1,3,850000
2,1012300674433870,1.41,13.78,95.85,3.77,7.98,54.50,49.87,2.54,1538.99,1069.01,69.46,0.454140,0.00,74000.000000,1

In [8]:
test_df.shape

(10000, 79)

In [9]:
# LabelEncoder for village_en_name and district_en_name

In [10]:
feature_cols = [
    # 'zipcode',
    'perc_of_house_with_6plus_room',
    'women_15_19_mothers_or_pregnant_at_time_of_survey',
    'perc_of_pop_living_in_hh_electricity',
    'perc_households_with_pucca_house_that_has_more_than_3_rooms',
    'mat_roof_metal_gi_asbestos_sheets',
    'perc_of_wall_material_with_burnt_brick',
    'households_with_improved_sanitation_facility',
    'perc_households_do_not_have_kcc_with_the_credit_limit_of_50k',
    'total_geographical_area_in_hectare',
    'net_agri_area_in_hectare',
    'perc_net_agri_area_in_hectare',
    'land_holding_index_source',
    'road_density',
    'avg_disbursement_amount_bureau',
    'kharif_seasons_cropping_density_2022',
    'rabi_seasons_cropping_density_2022',
    'kharif_seasons_agricultural_performance_2022',
    'rabi_seasons_agricultural_performance_2022',
    'kharif_seasons_agricultural_score_2022',
    'rabi_seasons_agricultiral_score_2022',
    'kharif_seasons_irrigated_area_2022',
    'rabi_seasons_irrigated_area_2022',
    'kharif_season_avg_groundwater_thickness_cm_2022',
    'rabi_season_avg_groundwater_thickness_cm_2022',
    'kharif_season_avg_groundwater_replenishment_rate_cm_2022',
    'rabi_season_avg_groundwater_replenishment_rate_cm_2022',
    'kharif_season_avg_rainfall_mm_2022',
    'rabi_season_avg_rainfall_mm_2022',
    'kharif_average_rainfall_2022_2021_growth',
    'rabi_average_rainfall_2022_2021_growth',
    'rabi_average_rainfall_2021_2020_growth',
    'kharif_season_groundwater_replenishment_rate_2022_2021_growth',
    'rabi_season_groundwater_replenishment_rate_2022_2021_growth',
    'kharif_season_groundwater_replenishment_rate_2021_2020_growth',
    'rabi_season_groundwater_replenishment_rate_2021_2020_growth',
    'kharif_groundwater_thickness_2022_2021_growth',
    'rabi_groundwater_thickness_2022_2021_growth',
    'kharif_groundwater_thickness_2021_2020_growth',
    'rabi_groundwater_thickness_2021_2020_growth',
    'kharif_season_agricultural_performance_2022_2021_growth',
    'rabi_season_agricultural_performance_2022_2021_growth',
    'kharif_season_agricultural_performance_2021_2020_growth',
    'rabi_season_agricultural_performance_2021_2020_growth',
    'kharif_season_agriculture_score_2022_2021_growth',
    'rabi_season_agriculture_score_2022_2021_growth',
    'kharif_season_agriculture_score_2021_2020_growth',
    'rabi_season_agriculture_score_2021_2020_growth',
    'state',
    'region',
    'soil_type',
    'kharif_ambient_temperature_min_2022',
    'kharif_ambient_temperature_max_2022',
    'rabi_ambient_temperature_min_2022',
    'rabi_ambient_temperature_max_2022',
    'kharif_ambient_temperature_min_2021',
    'kharif_ambient_temperature_max_2021',
    'rabi_ambient_temperature_min_2021',
    'rabi_ambient_temperature_max_2021',
    'rabi_ambient_temperature_min_2020',
    'rabi_ambient_temperature_max_2020',
    'kharif_vill_cat_on_agri_params_2022',
    'rabi_vill_cat_on_agri_params_2022',
    'village_cat_on_socio_economic_params',
    'waterbody_reservoir',
    'waterbody_river',
    'waterbody_riverbank',
    'waterbody_water',
    'waterbody_wetland',
    'ecological_subzone',
    'zipcode',
    'no_of_active_loan_in_bureau',
    'non_agriculture_income',
    'active_loan_bins',
    'enc_ownership',
    'address_type',
    'village_en_name',
    'village_non_agri_income_bin',
    'village_total_land_for_agriculture_bin',
    'district_en_name',
    'district_non_agri_bin',
    'district_total_land_for_agriculture_bin',
]
target_col = ['Target_Variable/Total Income']

In [11]:
processing_util_path = PROCESSED_DIR / exp_name / 'processing_util.pkl'
with open(processing_util_path, 'rb') as f:
    processing_util = pickle.load(f)

In [12]:
categorical_cols = processing_util['categorical_columns']
# categorical_cols.remove('zipcode')
# categorical_cols.remove('village_en_name')
# categorical_cols.remove('district_en_name')

In [13]:
train_df[categorical_cols] = train_df[categorical_cols].astype('category')
# X_val[categorical_cols] = X_val[categorical_cols].astype('category')

In [14]:
processing_util['target_column']

['Target_Variable/Total Income']

In [15]:
feature_cols = processing_util['training_columns'][1:-1]
target_col = processing_util['target_column']
feature_cols, target_col

(['perc_of_house_with_6plus_room',
  'women_15_19_mothers_or_pregnant_at_time_of_survey',
  'perc_of_pop_living_in_hh_electricity',
  'perc_households_with_pucca_house_that_has_more_than_3_rooms',
  'mat_roof_metal_gi_asbestos_sheets',
  'perc_of_wall_material_with_burnt_brick',
  'households_with_improved_sanitation_facility',
  'perc_households_do_not_have_kcc_with_the_credit_limit_of_50k',
  'total_geographical_area_in_hectare',
  'net_agri_area_in_hectare',
  'perc_net_agri_area_in_hectare',
  'land_holding_index_source',
  'road_density',
  'avg_disbursement_amount_bureau',
  'kharif_seasons_cropping_density_2022',
  'rabi_seasons_cropping_density_2022',
  'kharif_seasons_agricultural_performance_2022',
  'rabi_seasons_agricultural_performance_2022',
  'kharif_seasons_agricultural_score_2022',
  'rabi_seasons_agricultiral_score_2022',
  'kharif_seasons_irrigated_area_2022',
  'rabi_seasons_irrigated_area_2022',
  'kharif_season_avg_groundwater_thickness_cm_2022',
  'rabi_season_av

In [16]:
X_train, X_val, y_train, y_val = train_test_split(train_df[feature_cols], train_df[target_col], test_size=0.2, shuffle=True)

In [17]:
feature_types = [
    'c' if x in categorical_cols else 'q' for x in feature_cols
]

# for feature, feat_type in zip(feature_cols, feature_types):
#     print(f"{feature} --> {feat_type}")

In [18]:
train = xgb.DMatrix(
    data=X_train[feature_cols],
    label=y_train[target_col],
    feature_names=feature_cols,
    feature_types=feature_types,
    enable_categorical=True,
)
val = xgb.DMatrix(
    data=X_val[feature_cols],
    label=y_val[target_col],
    feature_names=feature_cols,
    feature_types=feature_types,
    enable_categorical=True
)

# Model

In [19]:
# params = {
#     'objective': 'reg:squarederror',  # For regression
#     'eta': 0.1,                       # Learning rate
#     'max_depth': 6,                   # Maximum tree depth
#     'subsample': 0.8,                 # Subsample ratio
#     'colsample_bytree': 0.8,          # Column sample ratio
#     'eval_metric': 'rmse'             # Evaluation metric
# }

In [20]:
# params = {
#     'objective': 'reg:absoluteerror',
#     'booster': 'gbtree', 
#     'lambda': 5.0, 
#     'alpha': 0.5675616512211372, 
#     'max_depth': 4, 
#     'eta': 0.021858109782425545, 
#     'gamma': 5.0, 
#     'grow_policy': 'lossguide', 
#     'min_child_weight': 2, 
#     'subsample': 0.8717549435992915, 
#     'colsample_bytree': 0.9895769611718003
# }

In [21]:
def mape_eval(preds, dtrain):
    y_true = dtrain.get_label()
    # Avoid division by zero
    mask = y_true != 0
    y_true_safe = y_true[mask]
    preds_safe = preds[mask]
    
    mape = np.mean(np.abs((y_true_safe - preds_safe) / y_true_safe)) * 100
    # Return only name and value (not is_higher_better)
    return 'MAPE', mape


def mape_objective(y_pred, dtrain):
    y_true = dtrain.get_label()
    # Prevent division by zero
    mask = (y_true != 0)
    y_true_safe = y_true[mask]
    y_pred_safe = y_pred[mask]
    
    grad = np.sign(y_pred_safe - y_true_safe) / y_true_safe
    hess = 1 / np.abs(y_true_safe)
    
    # Handle zeros in y_true (optional)
    full_grad = np.zeros_like(y_pred)
    full_hess = np.zeros_like(y_pred)
    full_grad[mask] = grad
    full_hess[mask] = hess
    
    return full_grad, full_hess

    
params = {
    # 'objective': 'reg:squarederror',  # For regression
    'objective': 'reg:absoluteerror',  # For regression
    'eta': 0.01,                       # Learning rate
    'max_depth': 4,                   # Maximum tree depth
    'subsample': 0.8,                 # Subsample ratio
    'colsample_bytree': 0.8,          # Column sample ratio
    'gamma': 5.0,
    'lambda': 5.0
}

In [22]:
# Train the model
# You can specify the number of boosting rounds (trees)
num_rounds = 10000

In [23]:
# When training
model = xgb.train(
    # obj=mape_objective,
    params=params,
    dtrain=train,
    num_boost_round=num_rounds,
    evals=[(train, 'train'), (val, 'validation')],
    custom_metric=mape_eval,  # Custom evaluation function
    early_stopping_rounds=500,
    verbose_eval=100,
)

[0]	train-mae:698698.59356	train-MAPE:219.61215	validation-mae:513227.92366	validation-MAPE:34.24381
[100]	train-mae:623753.05613	train-MAPE:190.64047	validation-mae:442142.99971	validation-MAPE:27.81720
[200]	train-mae:587763.36788	train-MAPE:174.61716	validation-mae:407266.43098	validation-MAPE:25.74245
[300]	train-mae:559080.35983	train-MAPE:168.31984	validation-mae:379847.66727	validation-MAPE:24.91387
[400]	train-mae:545788.22056	train-MAPE:166.82608	validation-mae:367811.26082	validation-MAPE:24.57800
[500]	train-mae:540377.35916	train-MAPE:164.95364	validation-mae:363718.12333	validation-MAPE:24.41248
[600]	train-mae:535466.08386	train-MAPE:166.02928	validation-mae:359689.66428	validation-MAPE:24.29672
[700]	train-mae:529513.61617	train-MAPE:166.16531	validation-mae:354407.25737	validation-MAPE:24.19350
[800]	train-mae:523643.14550	train-MAPE:166.14296	validation-mae:349290.26049	validation-MAPE:24.11413
[900]	train-mae:520569.82913	train-MAPE:166.09225	validation-mae:346881.936

In [27]:
model_dir = MODELS_DIR / exp_name
os.makedirs(model_dir, exist_ok=True)

In [28]:
model_path = model_dir / 'income_regression_model.bst'
model.save_model(model_path)

/var/folders/09/3m09y2cx0cq_1x4gp8q0ng2m0000gn/T/ipykernel_39984/3333930455.py:2: UserWarning: [00:46:18] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  model.save_model(model_path)


In [30]:
load_model = xgb.Booster()
load_model.load_model(model_path)

In [40]:
# test_df = test_df.drop(labels=['FarmerID'], axis=1)
test_dmatrix = xgb.DMatrix(
    data=test_df
)

In [41]:
load_model.predict(test_dmatrix)

array([1278948.6 , 1057276.  , 1013896.44, ...,  567200.56,  623875.56,
        621693.44], shape=(10000,), dtype=float32)

In [42]:
sorted_by_values = dict(sorted(model.get_score().items(), key=lambda item: item[1]))

sorted_by_values

{'waterbody_riverbank': 1.0,
 'waterbody_reservoir': 3.0,
 'village_cat_on_socio_economic_params': 4.0,
 'waterbody_water': 5.0,
 'waterbody_river': 7.0,
 'soil_type': 23.0,
 'kharif_ambient_temperature_max_2022': 25.0,
 'region': 25.0,
 'kharif_ambient_temperature_min_2022': 29.0,
 'kharif_ambient_temperature_min_2021': 31.0,
 'kharif_ambient_temperature_max_2021': 35.0,
 'district_total_land_for_agriculture_bin': 35.0,
 'rabi_ambient_temperature_min_2022': 36.0,
 'rabi_season_avg_rainfall_mm_2022': 37.0,
 'rabi_season_avg_groundwater_thickness_cm_2022': 38.0,
 'kharif_groundwater_thickness_2022_2021_growth': 40.0,
 'rabi_ambient_temperature_min_2020': 40.0,
 'kharif_season_avg_groundwater_replenishment_rate_cm_2022': 41.0,
 'rabi_ambient_temperature_max_2021': 41.0,
 'rabi_season_avg_groundwater_replenishment_rate_cm_2022': 42.0,
 'district_non_agri_bin': 42.0,
 'kharif_groundwater_thickness_2021_2020_growth': 45.0,
 'rabi_ambient_temperature_min_2021': 46.0,
 'total_geographical_are